In [1]:
#Import libraries
import numpy as np
import scipy as sp
from scipy import signal
import math
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, lfilter, freqz

In [2]:
#Input the test number (ref: Test Records)

#Test number on the record
test_number = 61
test_id = 'NAM' #Test identifier (NAM or Foam)
test_type = 'NAM4\Angled'

In [3]:
#Identify test from records
if test_id == 'NAM':
    test_record = pd.read_excel(r'F:\My Drive\Oxford 2022-24\NAM 3-4\NAM Test Records.xlsx')
elif test_id == 'Foam':
    test_record = pd.read_excel(r'F:\My Drive\Oxford 2022-24\DIHPB_Foam\Foam Test Records.xlsx')
    
record = pd.DataFrame(test_record, columns = ['Test No.','Sample Name','Thickness Avg','Firing Pressure','Impact Velocity','Projectile Mass','Save File'])
t_number = record['Test No.']
name = record['Sample Name']
vel = record['Impact Velocity']
thickness = record['Thickness Avg']
mass = record['Projectile Mass']
scope = record['Save File']

#Locator
m = list(t_number).index(test_number)

#Sample Name
test_name = name[m]
print(test_name)
#Scope data file name
scope_data = scope[m]

#File directory
if test_id =='NAM':
    up_directory = r'F:\My Drive\Oxford 2022-24\NAM 3-4\Test Data'
elif test_id =='Foam':
    up_directory = r'F:\My Drive\Oxford 2022-24\DIHPB_Foam\Test Data'
raw_data = os.path.join(up_directory,scope_data+'.'+'csv')
data = pd.read_csv(raw_data,skiprows=14)

#Create dataframe
#for bending 
df_CH = pd.DataFrame(data, columns = ['TIME','CH1','CH2','CH4'])
time = df_CH['TIME'] #assign time column
time = time.dropna()
v1 = df_CH['CH1'] #assign ch1 signal
v1 = v1.dropna()
v1 = v1-v1[0]
#for axial
v2 = df_CH['CH2'] #assign ch2 signal
v2 = v2.dropna()
v2 = v2-v2[0]

#Save directory
if test_id == 'NAM':
    save_dir = r'F:\My Drive\Oxford 2022-24\NAM 3-4\Analysed_Python'
elif test_id == 'Foam':
    save_dir = r'F:\My Drive\Oxford 2022-24\DIHPB_Foam\Analysed_Python'
analysed_data = os.path.join(save_dir,test_name)
if not os.path.exists(analysed_data):
    os.makedirs(analysed_data)

NAM4_B_30_4


In [4]:
#Test Parameters - *not to be changed unless re-calibrated*

b_factor = 2/6; #wheatstone bridge
amp = 500; #amplifier
g_factor = 1/2.1; #gauge
E_b = 210e9; #elastic modulus of bar (Steel)
r_b = (19.925/2)*1e-3; #radius of bar (m)
r_p = (11.91/2)*1e-3; #projectile radius (m)
r_a = (25.35/2)*1e-3; #anvil radius (m)
vel_p = vel[m]; #projectile velocity (m/s)
d_s = thickness[m]*1e-3; #sample thickness (m)
m_p = mass[m]*1e-3; #projectile mass (kg)
m_a = 239.4e-3  #anvil mass (kg)

In [5]:
#Isolate the Bending signal 
%matplotlib 
plt.figure(1)
plt.plot(time,v1)
#Click before and after the initial pulse to zoom in
x1 = plt.ginput(2) 
plt.close()
#We use time as the reference
t_1 =x1[0][0]
t_2 = x1[1][0]
#locate indices in the time array
i=0
while time[i]<t_1:
    i=i+1
i    
j=0
while time[j]<t_2:
    j=j+1
j
#Isolated pulse for both signal and time
zoom_v = v1[i:j]
zoom_v = zoom_v.reset_index(drop=True)
zoom_time = time[i:j]
zoom_time = zoom_time.reset_index(drop=True)

#Click for beginning and end of the pulse
plt.figure(2)
plt.plot(zoom_time,zoom_v)
x2 =plt.ginput(2)
plt.close()
t_begin =x2[0][0]
t_end =x2[1][0]
#locate indices in the time array
p=0
while zoom_time[p]<t_begin:
    p=p+1    
q=0
while zoom_time[q]<t_end:
    q=q+1
#Isolated pulse for both signal and time
new_v = -zoom_v[p:q]
new_v = new_v.reset_index(drop=True)
new_time = zoom_time[p:q]
new_time = new_time.reset_index(drop=True)



Using matplotlib backend: <object object at 0x00000237FF052F00>


In [6]:
#Isolate the AXIAL signal
#%matplotlib 
plt.figure(3)
plt.plot(time,v2)
#Click before and after the initial pulse to zoom in
x3 = plt.ginput(2) 
plt.close()
#We use time as the reference
t_3 =x3[0][0]
t_4 = x3[1][0]
#locate indices in the time array
k=0
while time[k]<t_3:
    k+=1
    
l=0
while time[l]<t_4:
    l+=1
#Isolated pulse for both signal and time
zoom_v2 = v2[k:l]
zoom_v2 = zoom_v2.reset_index(drop=True)
zoom_time2 = time[k:l]
zoom_time2 = zoom_time2.reset_index(drop=True)

#Click for beginning and end of the pulse
plt.figure(4)
plt.plot(zoom_time2,zoom_v2)
x4 =plt.ginput(2)
plt.close()
t_begin2 =x4[0][0]
t_end2 =x4[1][0]
#locate indices in the time array
A=0
while zoom_time2[A]<t_begin2:
    A+=1    
B=0
while zoom_time2[B]<t_end2:
    B+=1
#Isolated pulse for both signal and time
new_v2 = -zoom_v2[A:B]
new_v2 = new_v2.reset_index(drop=True)
new_time2 = zoom_time2[A:B]
new_time2 = new_time2.reset_index(drop=True)


In [7]:
#Parameter Calculations
#Axial strain and force
strain_x = b_factor*g_factor*new_v2/amp #calibration parameters
force_x = (np.pi*r_b**2)*E_b*strain_x #force on bar (E = stress/strain)
#Bending strain
strain_y = new_v/(500*6*2.1*4)

strain_x = strain_x-strain_x[0]
strain_y = strain_y-strain_y[0]

In [8]:
#Plots

%matplotlib 

fig, axs = plt.subplots(3,1)
plt.rcParams['axes.grid'] = True

#Axial strain
axs[0].plot(new_time2,strain_x, 'tab:red')
axs[0].set_ylabel('Axial Strain')
axs[0].set_xlabel('Time (s)')

#Axial force
axs[1].plot(new_time2,force_x, 'tab:blue')
axs[1].set_ylabel('Axial Force (N)')
axs[1].set_xlabel('Time (s)')

#Bending strain
axs[2].plot(new_time,strain_y, 'tab:orange')
axs[2].set_ylabel('Bending Strain')
axs[2].set_xlabel('Time (s)')

fig.tight_layout()

#Save Figure
filename = analysed_data + '\Angled_Measurements' + '.png'
plt.savefig(filename)


Using matplotlib backend: TkAgg


In [9]:
#Save dataset as csv file
#Test data
raw_dataset = pd.DataFrame({'Raw Time': time, 'Axial Voltage': v2, 'Bending Voltage': v1}) 
#Analysed data
analysed_dataset = pd.DataFrame({'Axial Time': new_time2,'Axial Strain': strain_x, 'Axial Force': force_x, 'Bending Time': new_time,'Bending Strain': strain_y})
#Combined data
all_dataset = pd.concat([raw_dataset, analysed_dataset], axis=1)

#Convert and save as csv
filename3 = analysed_data + '\\' + test_name + '.csv'
all_dataset.to_csv(filename3, index=False)

#For combined plot
if test_id == 'NAM':
    filename4 = os.path.join(r'F:\My Drive\Oxford 2022-24\NAM 3-4\Figures',test_type)
    if not os.path.exists(filename4):
        os.makedirs(filename4)
    savefilename = filename4 + '\\' + test_name + '.csv'
    all_dataset.to_csv(savefilename, index=False)
elif test_id == 'Foam':
    filename4 = os.path.join(r'F:\My Drive\Oxford 2022-24\DIHPB_Foam\Figures',test_name + '.csv')
    all_dataset.to_csv(filename4, index=False)

In [10]:
if test_id =='Foam':
    fig,ax1 =plt.subplots()
    ax1.plot(new_time[:-1],srate)
    ax1.set_xlabel('time (s)')
    ax1.set_ylabel('strain rate (/s)')
    ax1.tick_params('y')
    
    ax2 = ax1.twinx()
    ax2.plot(new_time[:-1],strain_s[:-1])
    ax2.set_ylabel('strain')
    ax2.tick_params('y')
    
    
    plt.show()